# <center>Estructura de Datos</center>
## <center>Universidad Nacional de Tres de Febrero</center>
## <center>Trabajo Práctico</center>

### Diagrama de clases

![diagrama_de_clases](diagrama.png)

### Análisis de las páginas seleccionadas

#### Cada página seleccionada (MercadoLibre, Rodo, Frávega, Garbarino) posee su clase que utiliza scrapy para bajar la información necesaria, ya que en el constructor de cada araña se debe especificar la búsqueda para iniciar la recuperación de datos. También en cada página, al buscar cada artículo, el título del mismo se escribe todo en minúscula para estandarizar futuras modificaciones.
#### En las tiendas "Rodo" y "Frávega" se convierte el precio del artículo en tokens, también para estandarizar ya que por ejemplo el precio de un artículo en la tienda "Rodo" aparece como "$ 50.500,00", eliminamos el signo pesos y la coma. Y en la tienda "Frávega" solamente eliminamos el signo pesos ya que el precio de sus articulos no tenian ",00" y además "inventamos" una url para cuando la búsqueda este vacía ya que sino esta misma página recupera todos los artículos que están en ella, esto no pasaría en las demás páginas. En la tienda "Garbarino" eliminamos algunos espacios que estaban de más en el título del artículo.Por último, en la tienda "MercadoLibre" no hubo más modificaciones que estandarizar el título.

#### Ejemplo de la url "inventada" en la tienda "Frávega":

### Decisiones de diseño

#### Decidimos  crear una interfaz gráfica (con tkinter) para manejar un menú de búsqueda mas cómodo, el mismo posee el tipo de búsqueda que se quiere realizar ("Predeterminado" : recuperará todos los artículos que se encuentren en la tienda seleccionada con respecto a la búsqueda inicial, "Frase exacta" : solo se mostraran aquellos artículos que sean igual a la búsqueda, "Todas las palabras" : aquellos artículos que posean en su título las palabras de la búsqueda, "Algunas de las palabras" : aquellos artículos que contengan alguna/s de las palabras de la búsqueda. En los dos últimos tipo de búsqueda se omiten los stopwords) y también el archivo de salida (CSV, HTML o JSON).
#### Creamos una clase "CargarArchivo" para pasar los datos cargados principalmente de un csv auxiliar (este mismo contiene los artículos cargados obtenidos por la/s araña/s anteriormente) a un diccionario para luego según el tipo de búsqueda que se quiere realizar, escribir estos datos en el archivo de salida seleccionado, con la fecha, hora y búsqueda. Al finalizar la búsqueda y al crearse el archivo de salida, se elimina el csv auxiliar ya que para nosotros luego de obtener los datos que queremos, no tiene importancia. También se cierra la interfaz gráfica, si se quiere realizar otra búsqueda, se deberá iniciar otra vez.

#### Ejemplo:

##### abrimos el archivo csv con los productos

In [3]:
productos = open ("rodo.csv")

In [4]:
#vemos que contiene el csv, en este caso "rodo.csv" seria nuestro csv auxiliar
for linea in productos:
    print(linea)
productos.close()

precio,titulo,link,categoria,pagina

1.249,panasonic auriculares rp-hs46pp-k,https://rodo.com.ar/panasonic-auriculares-rp-hs46pp-k.html,Panasonic Auriculares RP-HS46PP-K,Rodo

4.749,philips auricular shb3075bk supraural bluetooth c/mic negro,https://rodo.com.ar/philips-auricular-supraural-shb3075bk-bluetooth-c-mic-negro.html,Philips Auricular SHB3075BK SUPRAURAL Bluetooth c/Mic Negro,Rodo

1.699,sony auricular intrauditivo con mic. mdr-ex15apb negro,https://rodo.com.ar/sony-auricular-intrauditivo-con-mic-mdr-ex15apb-negro.html,Sony Auricular Intrauditivo con Mic. MDR-EX15APB Negro,Rodo

749,philips auriculares con micrÃ³fono taue101bk/00 negro,https://rodo.com.ar/philips-auriculares-con-microfono-taue101bk-00-negro.html,Philips Auriculares con micrÃ³fono TAUE101BK/00 Negro,Rodo

1.499,panasonic auriculares sports blanco rp-hs34ppw,https://rodo.com.ar/panasonic-auriculares-sports-rp-hs34ppw.html,Panasonic Auriculares Sports Blanco RP-HS34PPW,Rodo

4.050,noblex auriculares hp332b negro,h

In [8]:
import csv
from datetime import date
from datetime import datetime

class CargarArchivo:

    def __init__(self, nombre_archivo):

        #Asigno las tiendas donde buscar
        self.nombre_archivo = nombre_archivo
        self.contador=1

        #Creo un diccionario donde se van a guardar los productos cargados desde el .CSV
        self.diccionario={}


    def obtener_diccionario(self):

        if(self.nombre_archivo != None):
            archivo_a_leer = self.nombre_archivo
            with open(archivo_a_leer, "r", encoding="utf-8") as lugar_de_guardados:#agregue utf8

                lector = csv.DictReader(lugar_de_guardados, delimiter=",")
                for linea in lector:

                    self.diccionario['Elemento'+str(self.contador)]=[linea['titulo'],linea['precio'],linea['link'],
                    linea['categoria'], linea['pagina'], str(datetime.now())] #agrege str(datetime.now())] por que daba error json)

                    self.contador=self.contador+1

        return self.diccionario

##### Solo muestro la función "buscar_articulos" de la clase "Buscador", ya que en este caso, solo quiero buscar un artículo por "Frase exacta" y escribir_en_csv.

In [9]:
import json
from datetime import date
from datetime import datetime
import csv

class Buscador:

    def __init__(self, archivos_cargados, modo_de_busqueda, busqueda):

        self.archivos_cargados=archivos_cargados
        self.modo_de_busqueda=modo_de_busqueda
        self.busqueda=busqueda
        self.fecha= str(datetime.now().date())
        self.lista_new = []
        self.lista_return = []
        
    def buscar_articulos(self):
        
        if(self.modo_de_busqueda== "Frase exacta"):
            self.lista_new = []
            claves=list(self.archivos_cargados.keys())
            for elemento in claves:
                if(self.archivos_cargados[elemento][0] == self.busqueda):
                    self.lista_new.append(self.archivos_cargados[elemento])
            return self.lista_new
        
    def escribir_en_csv(self):
        self.lista_nuevo = self.buscar_articulos()
        
        with open(self.busqueda+' '+self.fecha+".csv", 'w',encoding="utf-8") as csvfile: #agregue utf8
            writer = csv.writer(csvfile, delimiter=';')
            writer.writerows(self.lista_nuevo)


In [10]:
cargar_archivo = CargarArchivo("rodo.csv")
diccionario = cargar_archivo.obtener_diccionario()

buscar = Buscador(diccionario, "Frase exacta", "noblex auriculares intrauditivos hpi 04 - negro")
buscar.escribir_en_csv()

In [12]:
producto_frase_exacta = open ("noblex auriculares intrauditivos hpi 04 - negro 2020-11-17.csv")

#Ahora utilizamos el print para ver que el artículo que buscamos antes sea igual al que obtenemos en el archivo de salida

for linea in producto_frase_exacta:
    print(linea)

noblex auriculares intrauditivos hpi 04 - negro;599;https://rodo.com.ar/noblex-auriculares-intrauditivos-hpi-04-negro.html;Noblex Auriculares intrauditivos HPI 04 - Negro;Rodo;2020-11-17 15:32:17.295095



